In [1]:
!pip install --upgrade gradio

In [2]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns

# Load the CSV files
csv_files = [
    r'C:\Users\Asus\OneDrive\Desktop\Flood prediction\Datasets\ASSAM-BARPETA.csv',
    r'C:\Users\Asus\OneDrive\Desktop\Flood prediction\Datasets\ASSAM-DHEMAJI.csv',
    r'C:\Users\Asus\OneDrive\Desktop\Flood prediction\Datasets\ASSAM-GOALPARA.csv',
    r'C:\Users\Asus\OneDrive\Desktop\Flood prediction\Datasets\ASSAM-LAKHIMPUR.csv',
    r'C:\Users\Asus\OneDrive\Desktop\Flood prediction\Datasets\UP-BALLIA.csv',
    r'C:\Users\Asus\OneDrive\Desktop\Flood prediction\Datasets\UP-BASTI.csv',
    r'C:\Users\Asus\OneDrive\Desktop\Flood prediction\Datasets\UP-GORAKHPUR.csv'
]

# Map districts
state_districts = {}
for file in csv_files:
    parts = file.split('-')
    state = parts[-2].split('\\')[-1].strip()
    district = parts[-1].split('.')[0].strip()
    if state not in state_districts:
        state_districts[state] = []
    state_districts[state].append(district)

print(state_districts)  # Debug print to check the content of state_districts

# Preprocess the data
def load_and_preprocess(file_path):
    data = pd.read_csv(file_path)
    data.columns = data.columns.str.strip()
    print(f"Columns in {file_path}: {data.columns.tolist()}")
    data['Dates'] = pd.to_datetime(data['Dates'], format='%d-%b-%y')

    if 'ACTUAL(mm)' in data.columns and 'NORMAL(mm)' in data.columns:
        data['Flood'] = (data['ACTUAL(mm)'] > data['NORMAL(mm)']).astype(int)
    else:
        raise KeyError(f"Required columns are missing in {file_path}")

    return data

# Calculate performance metrics
def train_and_evaluate(data):
    X = data[['NORMAL(mm)', 'ACTUAL(mm)']]
    y = data['Flood']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    return model, accuracy, report, conf_matrix

# Plotting data
def plot_data(data, district_name, flood_percentage):
    plt.figure(figsize=(10, 5))
    plt.plot(data['Dates'], data['ACTUAL(mm)'], label='Actual Rainfall (mm)')
    plt.plot(data['Dates'], data['NORMAL(mm)'], label='Normal Rainfall (mm)')
    plt.title(f'Rainfall Data for {district_name} & Flood % = {flood_percentage:.2f}')
    plt.xlabel('Date')
    plt.ylabel('Rainfall (mm)')
    plt.legend()
    plt.tight_layout()
    plt.savefig('rainfall_plot.png')
    plt.close()
    return 'rainfall_plot.png'

# Plot confusion matrix
def plot_confusion_matrix(conf_matrix, district_name):
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Flood', 'Flood'], yticklabels=['No Flood', 'Flood'])
    plt.title(f'Confusion Matrix for {district_name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.savefig('conf_matrix_plot.png')
    plt.close()
    return 'conf_matrix_plot.png'

# Calculate flood percentage
def calculate_flood_percentage(data):
    flood_percentage = (data['Flood'].sum() / len(data)) * 100
    return flood_percentage

# Load datasets and analyze
def load_and_analyze(state, district):
    file = f"C:\\Users\\Asus\\OneDrive\\Desktop\\Flood prediction\\Datasets\\{state}-{district}.csv"
    data = load_and_preprocess(file)

    model, accuracy, report, conf_matrix = train_and_evaluate(data)
    flood_percentage = calculate_flood_percentage(data)

    rainfall_plot = plot_data(data, district, flood_percentage)
    conf_matrix_plot = plot_confusion_matrix(conf_matrix, district)

    return str(accuracy), str(report), str(flood_percentage), rainfall_plot, conf_matrix_plot

# Gradio interface
with gr.Blocks() as iface:
    with gr.Tabs():
        with gr.TabItem("ASSAM"):
            district_assam = gr.Dropdown(label="Select District", choices=state_districts.get("ASSAM", []))
            output_assam_accuracy = gr.Textbox(label="Accuracy")
            output_assam_report = gr.Textbox(label="Classification Report")
            output_assam_flood_percentage = gr.Textbox(label="Flood Percentage")
            output_assam_plot = gr.Image(label="Rainfall Data")
            output_assam_conf_matrix = gr.Image(label="Confusion Matrix")

            district_assam.change(
                load_and_analyze,
                inputs=[gr.State("ASSAM"), district_assam],
                outputs=[output_assam_accuracy, output_assam_report, output_assam_flood_percentage, output_assam_plot, output_assam_conf_matrix]
            )

        with gr.TabItem("UTTAR PRADESH"):
            district_up = gr.Dropdown(label="Select District", choices=state_districts.get('UP', []))
            output_up_accuracy = gr.Textbox(label="Accuracy")
            output_up_report = gr.Textbox(label="Classification Report")
            output_up_flood_percentage = gr.Textbox(label="Flood Percentage")
            output_up_plot = gr.Image(label="Rainfall Data")
            output_up_conf_matrix = gr.Image(label="Confusion Matrix")

            district_up.change(
                load_and_analyze,
                inputs=[gr.State("UP"), district_up],
                outputs=[output_up_accuracy, output_up_report, output_up_flood_percentage, output_up_plot, output_up_conf_matrix]
            )

iface.launch()


{'ASSAM': ['BARPETA', 'DHEMAJI', 'GOALPARA', 'LAKHIMPUR'], 'UP': ['BALLIA', 'BASTI', 'GORAKHPUR']}
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Columns in C:\Users\Asus\OneDrive\Desktop\Flood prediction\Datasets\ASSAM-BARPETA.csv: ['Dates', 'NORMAL(mm)', 'ACTUAL(mm)']
Columns in C:\Users\Asus\OneDrive\Desktop\Flood prediction\Datasets\ASSAM-GOALPARA.csv: ['Dates', 'NORMAL(mm)', 'ACTUAL(mm)']
Columns in C:\Users\Asus\OneDrive\Desktop\Flood prediction\Datasets\UP-BALLIA.csv: ['Dates', 'NORMAL(mm)', 'ACTUAL(mm)']
